# Import modules

In [1]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
import datetime
import os

# Load data

In [3]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 32
epochs = 30
#(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces", image_size, batch_size, preprocess_input)
#(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace", image_size, batch_size, preprocess_input)
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/museumFaces_split", image_size, batch_size, preprocess_input, None, False, False)
(ffds_train, fftrain_batches, ffds_val, ffval_batches, ffds_test, fftest_batches, ffcount_classes) = ir.readData("/mimer/NOBACKUP/groups/snic2022-22-1091/FairFace_split", image_size, batch_size, preprocess_input, None, False, False)


Found 3916 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Count classes: (2052, 1864, 256, 233, 258, 234)
Found 78158 images belonging to 2 classes.
Found 9769 images belonging to 2 classes.
Found 9771 images belonging to 2 classes.
Count classes: (36736, 41422, 4592, 5177, 4592, 5179)


# VGG16 without reweighting

In [3]:
class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    base_model = m.build_model((224,224), "VGG")
    metric_list = m.metrics_list()

base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

In [4]:
utils.train_model(base_model, epochs, ds_train, train_batches, ds_val, val_batches, None, False)

Epoch 1/30
sc
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
123/123 [==============================] - 94s 394ms/step - loss: 0.6928 - accuracy: 0.5186 - true_positives: 68.0000 - true_negatives: 1963.0000 - false_positives: 89.0000 - false_negatives: 1796.0000 - true_positive_rate: 0.0365 - true_negative_rate: 0.9566 - false_positive_rate: 0.0434 - false_negative_rate: 0.9635 - positive_predicted_value: 0.4331 - false_discovery_rate: 0.5669 - negative_predicted_value: 0.5222 - false_omission_rate: 0.4778 - binary_demographic_parity_diff: -3602.0000 - demographic_parity: 157.0000 - binary_equalized_odds_diff: 0.9201 - binary_proportional_parity_diff: -0.9198 - proportional_parity: 0.0401 - binary_predictive_rate_parity_diff: -0.0891 - predictive_rate_parity: 0.4331 - binary_accuracy_parity_diff: 0.0373 - accuracy_parity: 0.5186 - false_negative_rate_parity_diff:

In [5]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

print("Name directory: " + dir_name)
utils.saveModel(base_model, dir_name + "/base_model.h5")

Make folder...
Name directory: 20221128_193133_Test


In [6]:
test_predict, test_labels, dir_name = ev.testModel(base_model, test_batches, dir_name)
ev.testModelWithThresholdChange(base_model, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
16/16 - 2s
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.7276422764227642
    
    True positives: 177
    False positives: 77
    
    True negatives: 181
    False negatives: 57
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.7564102564102564
    True negative rate tn/(tn+fp): 0.7015503875968992
    
    False negative rate fn/(tp+fn): 0.24358974358974358
    False positive rate fp/(tn+fp): 0.29844961240310075
    
    Positive predicted value tp/(tp+fp): 0.6968503937007874
    False discovery rate fp/(tp+fp): 0.3031496062992126
    
    Negative predicted value tn/(tn+fn): 0.7605042016806722
    False omission rate fn/(tn+fn): 0.23949579831932774
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 16
    Demographic parity tp+fp: 254
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp

# VGG16 with reweighting

In [16]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    reweight_model = m.build_model((224,224), "VGG")
    metric_list = m.metrics_list()

reweight_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [17]:
utils.train_model(reweight_model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

Epoch 1/30
sc
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1
123/123 [==============================] - 136s 759ms/step - loss: 0.6931 - accuracy: 0.4980 - true_positives_105: 1290.0000 - true_negatives_70: 660.0000 - false_positives_85: 1364.0000 - false_negatives_90: 602.0000 - true_positive_rate: 0.6818 - true_negative_rate: 0.3261 - false_positive_rate: 0.6739 - false_negative_rate: 0.3182 - positive_predicted_value: 0.4861 - false_discovery_rate: 0.5139 - negative_predicted_value: 0.5230 - false_omission_rate: 0.4770 - binary_demographic_parity_diff: 1392.0000 - demographic_parity: 2654.0000 - binary_equalized_odds_diff: -0.3557 - binary_proportional_parity_diff: 0.3555 - proportional_parity: 0.6777 - binary_predictive_rate_parity_diff: -0.0369 - predictive_rate_parity: 0.4861 - binary_accuracy_parity_diff: -0.0041 - accuracy_parity: 0.4980 - false_negative_

In [18]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

print("Name directory: " + dir_name)
utils.saveModel(reweight_model, dir_name + "/base_model_reweighted.h5")

Make folder...
Name directory: 20221128_195350_Test


In [19]:
test_predict, test_labels, dir_name = ev.testModel(reweight_model, test_batches, dir_name)
ev.testModelWithThresholdChange(reweight_model, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
16/16 - 2s
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.7012195121951219
    
    True positives: 179
    False positives: 92
    
    True negatives: 166
    False negatives: 55
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.7649572649572649
    True negative rate tn/(tn+fp): 0.6434108527131783
    
    False negative rate fn/(tp+fn): 0.23504273504273504
    False positive rate fp/(tn+fp): 0.35658914728682173
    
    Positive predicted value tp/(tp+fp): 0.6605166051660517
    False discovery rate fp/(tp+fp): 0.33948339483394835
    
    Negative predicted value tn/(tn+fn): 0.751131221719457
    False omission rate fn/(tn+fn): 0.248868778280543
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 50
    Demographic parity tp+fp: 271
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp))

# VGG16 transfer learning without reweighting

In [24]:
class_weight = mit.findClassWeights(fftrain_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    transfer_model = m.build_model((224,224), "VGG")
    metric_list = m.metrics_list()

transfer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

Weight for class 0: 1.06
Weight for class 1: 0.94
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [25]:
utils.train_model(transfer_model, epochs, ffds_train, fftrain_batches, ffds_val, ffval_batches, None, False)

Epoch 1/30
sc
2443/2443 [==============================] - 420s 157ms/step - loss: 0.5618 - accuracy: 0.6824 - true_positives_168: 32457.0000 - true_negatives_112: 20878.0000 - false_positives_136: 15917.0000 - false_negatives_144: 8906.0000 - true_positive_rate: 0.7847 - true_negative_rate: 0.5674 - false_positive_rate: 0.4326 - false_negative_rate: 0.2153 - positive_predicted_value: 0.6710 - false_discovery_rate: 0.3290 - negative_predicted_value: 0.7010 - false_omission_rate: 0.2990 - binary_demographic_parity_diff: 18590.0000 - demographic_parity: 48374.0000 - binary_equalized_odds_diff: -0.2173 - binary_proportional_parity_diff: 0.2379 - proportional_parity: 0.6189 - binary_predictive_rate_parity_diff: -0.0300 - predictive_rate_parity: 0.6710 - binary_accuracy_parity_diff: 0.3648 - accuracy_parity: 0.6824 - false_negative_rate_parity_diff: 0.2173 - false_positive_rate_parity_diff: -0.2173 - binary_negative_predictive_rate_parity_diff: 0.0300 - negative_predictive_rate_parity: 0.67

In [26]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

print("Name directory: " + dir_name)
utils.saveModel(transfer_model, dir_name + "/base_model_transfer.h5")

Make folder...
Name directory: 20221128_203944_Test


In [27]:
test_predict, test_labels, dir_name = ev.testModel(transfer_model, fftest_batches, dir_name)
ev.testModelWithThresholdChange(transfer_model, ffval_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
306/306 - 22s
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.8493501176952205
    
    True positives: 4405
    False positives: 698
    
    True negatives: 3894
    False negatives: 774
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.8505502992855763
    True negative rate tn/(tn+fp): 0.8479965156794426
    
    False negative rate fn/(tp+fn): 0.14944970071442362
    False positive rate fp/(tn+fp): 0.1520034843205575
    
    Positive predicted value tp/(tp+fp): 0.8632177150695669
    False discovery rate fp/(tp+fp): 0.13678228493043307
    
    Negative predicted value tn/(tn+fn): 0.8341902313624678
    False omission rate fn/(tn+fn): 0.16580976863753213
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): 435
    Demographic parity tp+fp: 5103
    
    Binary equalized odds diff (tp/(tp+fn))-(

# Transfer museum VGG16 with reweighting (22model untrainable layers)

In [7]:
class_weight = mit.findClassWeights(train_batches)
#dir_name = "20221128_203944_Test"

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = utils.loadModel(dir_name + "/base_model_transfer.h5", m.metrics_dict())
    metric_list = m.metrics_list()
    
utils.freezeCertainLayers(model, 20)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

print(model.summary())

Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:local

In [8]:
epochs = 20
utils.train_model(model, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

Epoch 1/20
sc
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
123/123 [==============================] - 90s 380ms/step - loss: 0.5751 - accuracy: 0.6910 - true_positives_101: 1325.0000 - true_negatives_66: 1381.0000 - false_positives_81: 671.0000 - false_negatives_86: 539.0000 - true_positive_rate: 0.7108 - true_negative_rate: 0.6730 - false_positive_rate: 0.3270 - false_negative_rate: 0.2892 - positive_predicted_value: 0.6638 - false_discovery_rate: 0.3362 - negative_predicted_value: 0.7193 - false_omission_rate: 0.2807 - binary_demographic_parity_diff: 76.0000 - demographic_parity: 1996.0000 - binary_equalized_odds_diff: -0.0378 - binary_proportional_parity_diff: 0.0194 - proportional_parity: 0.5097 - binary_predictive_rate_parity_diff: -0.0554 - predictive_rate_parity: 0.6638 - binary_accuracy_parity_diff: 0.3820 - accuracy_parity: 0.6910 - false_negative_rate_p

In [10]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

print("Name directory: " + dir_name)
utils.saveModel(model, dir_name + "/model_transfer_museum.h5")

Make folder...
Name directory: 20221128_211118_Test


In [11]:
test_predict, test_labels, dir_name = ev.testModel(model, test_batches, dir_name)
ev.testModelWithThresholdChange(model, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
16/16 - 2s
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.7601626016260162
    
    True positives: 172
    False positives: 56
    
    True negatives: 202
    False negatives: 62
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.7350427350427351
    True negative rate tn/(tn+fp): 0.7829457364341085
    
    False negative rate fn/(tp+fn): 0.26495726495726496
    False positive rate fp/(tn+fp): 0.21705426356589147
    
    Positive predicted value tp/(tp+fp): 0.7543859649122807
    False discovery rate fp/(tp+fp): 0.24561403508771928
    
    Negative predicted value tn/(tn+fn): 0.7651515151515151
    False omission rate fn/(tn+fn): 0.23484848484848486
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -36
    Demographic parity tp+fp: 228
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+

# Transfer fine-tuning museum VGG16

In [17]:
with strategy.scope():
    model_fine = utils.loadModel(dir_name + "/model_transfer_museum.h5", m.metrics_dict())
    metric_list = m.metrics_list()

model_fine.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

utils.freezeAllLayers(model, False)

In [18]:
epochs = 10
utils.train_model(model_fine, epochs, ds_train, train_batches, ds_val, val_batches, class_weight)

Epoch 1/10
sc
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
123/123 [==============================] - 131s 754ms/step - loss: 0.3342 - accuracy: 0.8713 - true_positives_284: 1644.0000 - true_negatives_186: 1768.0000 - false_positives_228: 285.0000 - false_negatives_242: 219.0000 - true_positive_rate: 0.8824 - true_negative_rate: 0.8612 - false_positive_rate: 0.1388 - false_negative_rate: 0.1176 - positive_predicted_value: 0.8523 - false_discovery_rate: 0.1477 - negative_predicted_value: 0.8898 - false_omission_rate: 0.1102 - binary_demographic_parity_diff: -58.0000 - demographic_parity: 1929.0000 - binary_equalized_odds_diff: -0.0213 - binary_proportional_parity_diff: -0.0148 - proportional_parity: 0.4926 - binary_predictive_rate_parity_diff: -0.0375 - predictive_rate_parity: 0.8523 - binary_accuracy_parity_diff: 0.7426 - accuracy_parity: 0.8713 - false_negative_

In [19]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

print("Name directory: " + dir_name)
utils.saveModel(model_fine, dir_name + "/model_transfer_museum_fine_tuned.h5")


Make folder...
Name directory: 20221128_212550_Test


In [20]:

test_predict, test_labels, dir_name = ev.testModel(model_fine, test_batches, dir_name)
ev.testModelWithThresholdChange(model_fine, val_batches, test_predict, test_labels, dir_name)

Testing Model
-------------
16/16 - 2s
Plot Histogram...
Plot ROC...
Plot Confusion matrix...
Plot Results...

    Test results (Without threshold change):
    -------------
    
    Accuracy: 0.758130081300813
    
    True positives: 178
    False positives: 63
    
    True negatives: 195
    False negatives: 56
    
    Test results metrics:
    ---------------------
    True positive rate tp/(tp+fn): 0.7606837606837606
    True negative rate tn/(tn+fp): 0.7558139534883721
    
    False negative rate fn/(tp+fn): 0.23931623931623933
    False positive rate fp/(tn+fp): 0.2441860465116279
    
    Positive predicted value tp/(tp+fp): 0.7385892116182573
    False discovery rate fp/(tp+fp): 0.26141078838174275
    
    Negative predicted value tn/(tn+fn): 0.7768924302788844
    False omission rate fn/(tn+fn): 0.22310756972111553
    
    
    Binary demographic parity diff (tp+fp)-(tn+fn): -10
    Demographic parity tp+fp: 241
    
    Binary equalized odds diff (tp/(tp+fn))-(tn/(tn+fp